#### Importing Required Library

In [ ]:
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
from torch import optim
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
from sklearn import metrics
import matplotlib.pyplot as plt
import torchvision

#### Mounting my drive for data access

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Providing datapath

In [ ]:
data_path = '/content/drive/MyDrive/Dataset'
batch_size = 32

#### Defining transforms and dataloader

In [ ]:
transform = transforms.Compose([
        transforms.Resize(256, interpolation=3),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

dataset_train = datasets.ImageFolder(os.path.join(data_path, "test"), transform=transform)

train_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

print(f"Data loaded with {len(dataset_train)} test imgs.")


Data loaded with 30 test imgs.


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


#### Setting up the model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 10, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Defining another 2D convolution layer
            nn.Conv2d(10, 15, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(15),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(15, 10, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
      )

        self.linear_layers = nn.Sequential(
            
          nn.Linear(10 * 28 * 28, 3)
      )

  # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

#### Load the saved drink detection model

In [ ]:
drink_model = Net()
drink_model.load_state_dict(torch.load('/content/drive/MyDrive/Dataset/drink_model.pth'))
if torch.cuda.is_available():
    drink_model = drink_model.cuda()
drink_model.eval()
print(drink_model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(15, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=7840, out_features=3, bias=True)
  )
)


#### Load the saved food detection model

In [ ]:
food_model = Net()
food_model.load_state_dict(torch.load('/content/drive/MyDrive/Dataset/food_model.pth'))
if torch.cuda.is_available():
    food_model = food_model.cuda()
food_model.eval()
print(food_model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(15, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=7840, out_features=3, bias=True)
  )
)


#### Define label to label text map

In [ ]:
drink_map = {0: "beer", 1: "soda", 2: "wine"}
food_map = {0: "fast_food", 1: "mexican food", 2: "steak"}
Matrix = np.zeros((3,3), dtype=np.int32)

#### Extract associations

In [ ]:
# getting predictions on test set and measuring the performance
r = []
for images,labels in train_loader:
    for i in range(len(labels)):
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        img = images[i].view(1, 3, 224, 224)
        with torch.no_grad():
            drink = drink_model(img)
            food = food_model(img)


        ps_drink = torch.exp(drink)
        probab_drink = list(ps_drink.cpu()[0])
        pred_label_drink = probab_drink.index(max(probab_drink))

        ps_food = torch.exp(food)
        probab_food = list(ps_food.cpu()[0])
        pred_label_food = probab_food.index(max(probab_food))
        r.append([drink_map[pred_label_drink], food_map[pred_label_food]])
        Matrix[pred_label_drink][pred_label_food] = Matrix[pred_label_drink][pred_label_food] + 1

#### Display the association matrix

In [ ]:
import pandas as pd
col_list = ['drink']*4
for key, val in food_map.items():
  col_list[key+1] = val

tabel = pd.DataFrame(columns=col_list)
for i in range(3):
  temp = []
  temp.append(drink_map[i])
  for j in range(3):
    temp.append(Matrix[i][j])
  tabel.loc[len(tabel)] = temp
tabel

,drink,fast_food,mexican food,steak
0,beer,13,3,3
1,soda,2,2,3
2,wine,2,1,1
